# Preprocess: ground truth

aim: create the image files with the ground truth (only the signal and only the part that corresponds to what was printed)

input: Reads .mat files

plots the corresponding 12 lead signal

output: saves the plot as an image

### Imports

In [ ]:
from matplotlib import pyplot as plt
import scipy.io
import glob
import os
import zipfile
import numpy as np
import cv2
from sklearn import preprocessing
from PIL import Image

In [ ]:
# define: results_path & data_path

### Load .mat files

In [ ]:
#load ecg from matlab files
os.chdir(data_path+'/mat')

# Get a list for .mat files in current folder
mat_files = glob.glob('*.mat')

# List for stroring all the data
alldata = {}

# Iterate mat files
for fname in mat_files:
    # Load mat file data into data.
    data = scipy.io.loadmat(fname+'.mat')
    # Append data to the list
    alldata[fname] = data['ECG']

In [ ]:
#there should be 90 total
len(alldata)

90

In [ ]:
datanorm = [0]*12
for i in range(12):
  datanorm[i] = 7.5*preprocessing.normalize([im[i]])

### func plot ecg

In [ ]:
!pip install ecg-plot

In [ ]:
import ecg_plot

li = ['I', 'II', 'III','aVR', 'aVL', 'aVF','V1', 'V2', 'V3','V4', 'V5', 'V6']

for id_ in names:
  ecg = alldata[id_]['data'][0][0]

  datanorm = ecg
  shape = []
  for i in range(0,12):
      datanorm[i,:] = 7.5*preprocessing.normalize([ecg[i,:]])
      tmp = datanorm.shape
      shape.append(tmp[1])
  min_shape = np.amin(shape)

  if min_shape > 20000: 
    ecg_plot.plot(ecg[0:12,0:20000], sample_rate=500, title = '', style = 'bw', columns = 1, row_height= 3, show_separate_line  = False,show_lead_name = False, show_grid = False)
    ecg_plot.save_as_jpg(id_,results_path)

  else: 
    ecg_plot.plot(ecg, sample_rate=500, title = '', style = 'bw', columns = 1, row_height= 3, show_separate_line  = False, show_lead_name = False, show_grid = False)
    ecg_plot.save_as_jpg(id_,results_path)

### Load digital (gt) signals

In [ ]:
# matlab plotted
folder = results_path

imgs_mat = {}
for filename in os.listdir(folder):
  img = cv2.imread(os.path.join(folder,filename))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  imgs_mat[filename.strip('.jpg')] = img

print("No. of images = ", len(imgs_mat))

No. of images =  90


### get sizes, proportions of what was printed

In [ ]:
#SAVE THE SIZES OF WHAT HAS BEEN PRINTED


#FITTED
fh = (502, 720) #all width appears but only a part of the height
fw = (518, 742) #all height appears but only a part of width

for id_ in list(imgs_mat.keys()):
  img = imgs_mat[id_]
  h,w = img.shape
  
  fit = fh
  if w > 720: # if the picture is too wide it fits only a part (middle)
    fit = fw
  
  ext_h = int((h - fit[0])/2)
  ext_w = int((w - fit[1])/2)
  
  cropped = img[ext_h:(h-ext_h), ext_w:(w-ext_w)]

  im = Image.fromarray(cropped)
  im.save(data_path+'cropped_mat/'+id_+'.jpg')